#berita berdasarkan Topik


##Import libary


In [15]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Load dataset (kompas_scraping_results.csv)

In [3]:
data = pd.read_csv('/content/drive/MyDrive/STKI_DM/kompas_scraping_results.csv')

## Data Cleaning

In [4]:
def clean_text(text):
    # Removing unwanted characters, punctuation, and converting to lowercase
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    return text

##Apply cleaning on 'tittle' and 'Content'

In [5]:
data['Title'] = data['Title'].apply(clean_text)
data['Content'] = data['Content'].apply(clean_text)

#Data Preprocessing


## Combine 'tittle' and 'Content' for text-based features

In [6]:
data['combined_text'] = data['Title'] + ' ' + data['Content']

##Encode 'category' column

In [7]:
label_encoder = LabelEncoder()
data['category_encoded'] = label_encoder.fit_transform(data['Category'])

##TF-IDF Vectorization

In [8]:
vectorizer = TfidfVectorizer(max_features=5000)  # Use top 5000 features
X_text = vectorizer.fit_transform(data['combined_text']).toarray()

##Combine text features and category encoding

In [9]:
X = np.hstack((X_text, data['category_encoded'].values.reshape(-1, 1)))
y = data['category_encoded']

##Split data into training and testing sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##KNN Classifier

In [11]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

##Predictions

In [12]:
y_pred = knn.predict(X_test)

##Evaluation

In [14]:
from sklearn.metrics import classification_report

# Mendapatkan label unik dalam data uji
unique_labels = np.unique(y_test)

# Evaluasi menggunakan classification_report
print("Classification Report:\n", classification_report(
    y_test, y_pred, labels=unique_labels, target_names=label_encoder.inverse_transform(unique_labels)
))


Classification Report:
               precision    recall  f1-score   support

 ADVERTORIAL       1.00      1.00      1.00         1
        BOLA       1.00      1.00      1.00         3
        FOOD       1.00      1.00      1.00         3
      GLOBAL       1.00      1.00      1.00         1
        HYPE       1.00      1.00      1.00         5
   LIFESTYLE       1.00      1.00      1.00         1
       MONEY       1.00      1.00      1.00        10
        NEWS       1.00      1.00      1.00        22
    OTOMOTIF       1.00      1.00      1.00         8
    PROPERTI       1.00      1.00      1.00         1
        PROV       1.00      1.00      1.00         4
    REGIONAL       1.00      1.00      1.00        19
       TEKNO       1.00      1.00      1.00         5
      TRAVEL       1.00      1.00      1.00         2
        TREN       0.80      1.00      0.89         4
        UMKM       0.00      0.00      0.00         1

    accuracy                           0.99        90
  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##Menyimpan Model


In [16]:
with open('knn_model.pkl', 'wb') as file:
    pickle.dump(knn, file)